In [ ]:
from moldf import read_mol2
import os

PATH = '/home/skrhakv/Near-Hit-Scoring/data/scPDB'

In [ ]:
mapping = {'Aba': 'A', 'Ace': 'X', 'Acr': 'X', 'Ala': 'A', 'Aly': 'K', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cas': 'C',
           'Ccs': 'C', 'Cme': 'C', 'Csd': 'C', 'Cso': 'C', 'Csx': 'C', 'Cys': 'C', 'Dal': 'A', 'Dbb': 'T', 'Dbu': 'T',
           'Dha': 'S', 'Gln': 'Q', 'Glu': 'E', 'Gly': 'G', 'Glz': 'G', 'His': 'H', 'Hse': 'S', 'Ile': 'I', 'Leu': 'L',
           'Llp': 'K', 'Lys': 'K', 'Men': 'N', 'Met': 'M', 'Mly': 'K', 'Mse': 'M', 'Nh2': 'X', 'Nle': 'L', 'Ocs': 'C',
           'Pca': 'E', 'Phe': 'F', 'Pro': 'P', 'Ptr': 'Y', 'Sep': 'S', 'Ser': 'S', 'Thr': 'T', 'Tih': 'A', 'Tpo': 'T',
           'Trp': 'W', 'Tyr': 'Y', 'Unk': 'X', 'Val': 'V', 'Ycm': 'C', 'Sec': 'U', 'Pyl': 'O', 'Mhs': 'H', 'Snm': 'S',
           'Mis': 'S', 'Seb': 'S', 'Hic': 'H', 'Fme': 'M', 'Asb': 'D', 'Sah': 'C', 'Smc': 'C', 'Tpq': 'Y', 'Onl': 'X',
           'Tox': 'W', '5x8': 'X', 'Ddz': 'A'}


def three_to_one(three_letter_code):
    if three_letter_code[0].upper() + three_letter_code[1:].lower() not in mapping:
        return 'X'
    return mapping[three_letter_code[0].upper() + three_letter_code[1:].lower()]

sequences = {}
for directory in os.listdir(PATH):
    site_mol_file = read_mol2(mol2_file=f'{PATH}/{directory}/site.mol2')
    site = site_mol_file['ATOM']
    binding_residues = site[site['atom_name'] == 'CA']['subst_name'].tolist()

    protein_mol_file = read_mol2(mol2_file=f'{PATH}/{directory}/protein.mol2')
    protein = protein_mol_file['ATOM']
    all_residues = protein[protein['atom_name'] == 'CA']['subst_name'].tolist()
    sequence = ''
    binding_residues_arr = []

    for i, res in enumerate(all_residues):
        sequence += three_to_one(res[:3])
        if res in binding_residues:
            binding_residues_arr.append(f'{three_to_one(res[:3])}{str(i)}')

    sequences[directory] = (sequence, binding_residues_arr)

In [48]:
import pandas as pd

csv_path = '/home/skrhakv/Near-Hit-Scoring/data/input/proteins_SI_90.csv'
proteins_si = pd.read_csv(csv_path, low_memory=False)

# split entries like "1iki_A" into pdb_id and chain
proteins_si[['pdb_id', 'chain']] = proteins_si.iloc[:, 0].str.split('_', expand=True)
proteins_90_SI = proteins_si['pdb_id'].tolist()

proteins_added_90_SI = set()
with open('/home/skrhakv/Near-Hit-Scoring/data/input/scPDB_all.csv', 'w') as f:
    with open('/home/skrhakv/Near-Hit-Scoring/data/input/scPDB_90_SI.csv', 'w') as f90:
        for protein_id, (sequence, binding_residues) in sequences.items():
            pdb_id, chain_id = protein_id.split('_')
            line = f'{pdb_id};{chain_id};UNKNOWN;{" ".join(binding_residues)};{sequence}\n'
            f.write(line)
            if pdb_id in proteins_90_SI and pdb_id not in proteins_added_90_SI:
                f90.write(line)
                proteins_added_90_SI.add(pdb_id)